# Needle Segmentation and Localization

In segmentation our goal is to extract a specific object or group of objects from an image. In this case the object is needle.

In this work a binary classification model based on convolutional nural Network (CNNs) is proposed for needle segmentation and localization. We need to extract the needle from the back ground of image which is a binary classification problem. To do this we need to classify each pixel on the image as either being part of a needle or not. The target values are given to us as a black and white mask.

With the aim of performing segmentation I would be using the UNet architecture to classify image pixels as belongong to a needle or background. UNet is a very popular image segmentation archutecture initially designed for biomedical image processing but later adapted by practitionars from other fields as well. 

# Convolutional Neural Networks

Medical image segmentation can be formulated as a pixel_level
classification problem which can be solved by convolutional
neural networks
In this work needle segmentation and localization is a binary classification problem based on CNNs.
The deep network architecture is composed of sequential convolutional layers. At each convolutional
layer l, the input feature map (image) is convolved by a set of K kernels to generate a new feature map. A non-linear activation function is then applied to this feature map to generate the output which is the input for the next layer.The concatenation of the feature maps at each layer provides a combination of patterns to the network, which become increasingly complex for deeper layers.

Training of the CNN is usually done through several iterations of stochastic gradient
descent (SGD), in which several samples of training data (abatch) is processed by the network. At each iteration, based on the calculated loss the network parameters (kernel weights and biases) are optimized by SGD in order to decrease the loss.

The CNN proposed in this paper is a 2D U_Net. U_Net for segmentation usually consist of an encoder (contracting)path and a decoder (expanding) path. The encoder path consists of several convolutional layers followed by activation functions with max-pooling layers on selected feature maps.
The encoder path decreases the resolution of the feature maps(images) by computing the maximum of small patches of units of the feature maps. However, good resolution is critical for accurate segmentation, therefore in the decoder path, upsampling is performed to restore the initial resolution, but the feature maps are concatenated to keep the computation and memory requirements tractable.
As a result of multiple convolutional
layers and max-pooling operations the feature maps
are reduced and the intermediate layers of an FCN become
successively smaller. Therefore, following the convolutions,
an FCN uses inverse convolutions (or backward convolutions)
to up-sample the intermediate layers until the input resolution
is matched

# importing all basic python libraries

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
import sys
import glob
import pandas as pd
import numpy as np
from skimage import io
from skimage.transform import resize
# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')
 
%matplotlib inline    
print(tf.version.VERSION)

# Read the Images & Data Preprocessing

I have 2d RBG data set (2D, 3 channels) from strio vision camera and my goal is segmention of image pixle as a needle or backgrand.Each image is a matrix with values that range between 0 and 255. In this section I read imges cv2.imread from needle_seg folder one by one and put them in needle image vector. I crop the image margin and then resize the image (512,512) which is the size I chose as size of input image for network.

In [ ]:
needle_img = []
needle_crop = []

for img in glob.glob("../needle_seg/*.png"):
    n = cv2.imread(img)
    n = cv2.cvtColor(n,cv2.COLOR_BGR2RGB)
    cropped = n[50:(n.shape[0]-100),200:(n.shape[1]-300)]
    needle_img.append(n)
    needle_crop.append(cropped)

In [ ]:
len(needle_crop)

In [ ]:
import random
image_indx = random.sample(range(488), 12)
image_indx

fig, ax = plt.subplots(figsize=(15,9),ncols=4, nrows=3)

for row_num in range(4):
        
    img = ax[0][row_num].imshow(needle_img[image_indx[0+row_num]],cmap='gray')
    
        
    img = ax[1][row_num].imshow(needle_img[image_indx[1+row_num]],cmap='gray')
    
    img = ax[2][row_num].imshow(needle_img[image_indx[2+row_num]],cmap='gray')
                    
plt.show()

In [ ]:

fig, ax = plt.subplots(figsize=(15,9),ncols=4, nrows=3)

for row_num in range(4):
        
    img = ax[0][row_num].imshow(needle_crop[image_indx[0+row_num]],cmap='gray')
    
        
    img = ax[1][row_num].imshow(needle_crop[image_indx[1+row_num]],cmap='gray')
    
    img = ax[2][row_num].imshow(needle_crop[image_indx[2+row_num]],cmap='gray')
                    
plt.show()

# creating ground truth

Using deep learning technieques for image segmentation requier ground truth information as annotated images to train a modle. I annotaded needle images for deep learning  using free toole apeer annotate from this www.apeer.com/annotate to create labeled image for training some of the needle images.

The mask is a 2D matrix. It contains only two values that represent either black or white.


So we have just read 51 images and coresponiding ground truth and consider them as a training set.

In [ ]:
import glob
cv_crop = []
for img in glob.glob("../needle_crop1/*.png"):
    
    # read the image using skimage.io and convert to RGB with
    n = io.imread(img)
    n = cv2.cvtColor(n,cv2.COLOR_BGR2RGB)
    n = cv2.cvtColor(n,cv2.COLOR_RGB2GRAY)
    
     # resize the image
    n = resize(n, (512, 512), mode='constant', preserve_range=True)
    n = n/255
    # use np.expand dims to add a channel axis so the shape becomes (IMG_HEIGHT, IMG_WIDTH, 1)
    n = np.expand_dims(n, axis=-1)
    
    cv_crop.append(n)
    
cv_crop[1].shape

In [ ]:
len(cv_crop)

In [ ]:
import glob
cv_mask = []
for img in glob.glob("../needle_crop_mask/*.ome.tiff"):
    
    # read the image using skimage
    n = io.imread(img)
    
    # resize the image
    n = resize(n, (512, 512), mode='constant', preserve_range=True)
    #n= n*255
    
    # use np.expand dims to add a channel axis so the shape becomes (IMG_HEIGHT, IMG_WIDTH, 1)
    n = np.expand_dims(n, axis=-1)
    
    cv_mask.append(n)
    
cv_mask[1].shape    

In [ ]:
cv_crop1 = []

for img in glob.glob("../Untitled Folder/*.png"):
    
    # read the image using skimage
    n = cv2.imread(img)
    n = cv2.cvtColor(n,cv2.COLOR_BGR2RGB)
    n = cv2.cvtColor(n,cv2.COLOR_RGB2GRAY)
    
    n = n[50:(n.shape[0]-100),200:(n.shape[1]-300)]
    
      # resize the image
    n = resize(n, (512, 512), mode='constant', preserve_range=True)
    n = n/255
    
    # use np.expand dims to add a channel axis so the shape becomes (IMG_HEIGHT, IMG_WIDTH, 1)
    n = np.expand_dims(n, axis=-1)
        
    cv_crop1.append(n)
    
cv_crop1[1].shape
    

In [ ]:
X_train = np.array(cv_crop)
y_train = np.array(cv_mask)
X_test = np.array(cv_crop1)

In [ ]:
X_train.shape

# Needle and ground truth image

Now we will have a look at some random images and its ground truth

In [ ]:
fig, ax = plt.subplots(figsize=(15,7),ncols=4, nrows=2)

for row_num in range(4):
    
    image_indx = np.random.randint(51)

    img = ax[0][row_num].imshow(X_train[image_indx][:,:,0],cmap='gray')
    ax[0][row_num].title.set_text('Needle_img')
    fig.colorbar(img, ax=ax[0][row_num])


    mask = ax[1][row_num].imshow(y_train[image_indx][:,:,0],cmap='gray')
    ax[1][row_num].title.set_text('Ground_truth')
    fig.colorbar(mask, ax=ax[1][row_num])
    
plt.show()

The image and its associated mask have the same shape and both are 2D. The image has pixel values in the range 0 to 1. The mask has pixel values that are either 0 or 1. 0 is black and 1 is white. 

# Data augmentation


In [ ]:
agment1_X_train = np.copy(X_train)
agment1_y_trian = np.copy(y_train)

for i in range(0,50):
    n = random.randint(0,400)
    agment1_y_trian[i][n:n+20]=0
    agment1_X_train[i][n:n+20]=1/2

In [ ]:
agment_X_train = np.copy(X_train)
agment_y_trian = np.copy(y_train)

for i in range(0,50):
    n = random.randint(0,400)
    agment_y_trian[i][n:]=0
    agment_X_train[i][n:]=1/2

In [ ]:
X_train = np.concatenate((X_train, agment1_X_train), axis=0)
y_train = np.concatenate((y_train, agment1_y_trian), axis=0)

In [ ]:
X_train.shape

In [ ]:
fig, ax = plt.subplots(figsize=(15,7),ncols=4, nrows=2)

for row_num in range(4):
    
    image_indx = np.random.randint(51)

    img = ax[0][row_num].imshow(agment1_X_train[image_indx][:,:,0],cmap='gray')
    ax[0][row_num].title.set_text('Needle_img')
    fig.colorbar(img, ax=ax[0][row_num])


    mask = ax[1][row_num].imshow( agment1_y_trian[image_indx][:,:,0],cmap='gray')
    ax[1][row_num].title.set_text('Ground_truth')
    fig.colorbar(mask, ax=ax[1][row_num])
    
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,7),ncols=4, nrows=2)

for row_num in range(4):
    
    image_indx = np.random.randint(51)

    img = ax[0][row_num].imshow(agment_X_train[image_indx][:,:,0],cmap='gray')
    ax[0][row_num].title.set_text('Needle_img')
    fig.colorbar(img, ax=ax[0][row_num])


    mask = ax[1][row_num].imshow(agment_y_trian[image_indx][:,:,0],cmap='gray')
    ax[1][row_num].title.set_text('Ground_truth')
    fig.colorbar(mask, ax=ax[1][row_num])
    
plt.show()

## Intersection-Over-Union (IoU, Jaccard Index)

the IoU is the area of overlap between the predicted segmentation and the ground truth divided by the area of union between the
predicted segmentation and the ground truth
For binary (two classes) or multi-class segmentation, the mean IoU of the image is calculated by taking the IoU of each class and averaging them. (It’s implemented slightly differently in code).

In [ ]:
from tensorflow.keras import backend as K

def iou_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true)+K.sum(y_pred)-intersection
    iou =(intersection + smooth) / (union + smooth)
    return iou

In [ ]:
def iou_coef_loss(y_true, y_pred):
    return -iou_coef(y_true, y_pred)

# Network Architecture

training a deep learning algorithm for segmentation, here I use 2d U-Net arhitecture. U-Net is a convolutional neural network that was developed for biomedical image segmentation. It was designed to give good results when only a small number of training images are available. It was also designed to run fast.

The network architecture consists of a contracting path (left side) and an expansive path (right side).
The contracting path follows the typical architecture of a convolutional network. It consists of the repeated application of two 3x3 convolutions (unpadded convolutions), each followed by a rectified linear unit (ReLU) and a 2x2 max pooling operation with stride 2 for downsampling. At each downsampling step we double the number of feature channels.

Every step in the expansive path consists of an upsampling of the feature map followed by a 2x2 convolution (\up-convolution") that halves the number of feature channels, a concatenation with the correspondingly cropped feature map from the contracting path, and two 3x3 convolutions, each followed by a ReLU.

The cropping is necessary due to the loss of border pixels in every convolution. At the final layer a 1x1 convolution is used to map each 16 component feature vector to the desired number of classes. In total the network has 33 convolutional layers.

 
Because we treat this as a binary classification problem, we will use the Sigmoid activation function in the last layer of the neural network. We will also use the binary_crossentropy loss function. We will train the network using the images as the input and the masks as the target.

In [ ]:
IMG_HEIGHT= 512
IMG_WIDTH = 512
IMG_CHANNELS = 1



# Build U-Net model
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

#  CONTRACTION PATH
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.2)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c5)
p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c5)


c6 = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p5)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c6)
p6 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c6)


c7 = tf.keras.layers.Conv2D(1024, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(p6)
c7 = tf.keras.layers.Dropout(0.3)(c7)
c7 = tf.keras.layers.Conv2D(1024, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c7)


# EXPANTION PATH

u8 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c6])
c8 = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.2)(c8)
c8 = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c5])
c9 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.2)(c9)
c9 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c9)

u10 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c9)
u10 = tf.keras.layers.concatenate([u10, c4])
c10 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u10)
c10 = tf.keras.layers.Dropout(0.2)(c10)
c10 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c10)

u11 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c10)
u11 = tf.keras.layers.concatenate([u11, c3])
c11 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u11)
c11 = tf.keras.layers.Dropout(0.2)(c11)
c11 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c11)

u12 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c11)
u12 = tf.keras.layers.concatenate([u12, c2])
c12 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u12)
c12 = tf.keras.layers.Dropout(0.1)(c12)
c12 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c12)


u13 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c12)
u13 = tf.keras.layers.concatenate([u13, c1], axis=3)
c13 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(u13)
c13 = tf.keras.layers.Dropout(0.1)(c13)
c13 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                            padding='same')(c13)

#Because we treat this as a binary classification problem, we will use the Sigmoid activation function
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c13)

#We will train the network using the images as the input and the masks as the target.
# We will also use the binary_crossentropy loss function.
model1 = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model1.compile(optimizer='adam', loss= [iou_coef_loss], metrics= [iou_coef])
model1.summary()


In [ ]:
tf.keras.utils.plot_model(model1, show_shapes=True)

In [ ]:
import visualkeras
visualkeras.layered_view(model1 , to_file='output.png')

# train model

During training of the proposed network, we aimed to minimize a loss function that measures the quality of the segmentation on the training examples. 
We will let Keras automatically create a 10% validation set during training.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.1)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True)
# This callback will stop the training when there is no improvement in
# the loss for three consecutive epochs.
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
history = model1.fit(X_train, y_train,validation_data=(X_valid,y_valid), batch_size=16, epochs=25, callbacks=[cp_callback])



In [ ]:
len(history.history['loss'])

In [ ]:
# Plot history: MAE

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure()
plt.plot(history.epoch, loss, 'r', label='Training loss')
plt.plot(history.epoch, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epoch')
plt.ylabel('loss Value')
#plt.ylim([0, 1])
plt.legend()
plt.show()



# Comparing Grand truth with Predicted Mask 

we made predictions on X_train and X_test using the trained model 
When Predict is run, the model will output a 2D mask. Each pixel in the predicted mask represents the probability that pixel is part of a needle. We need to convert these probabilities to binary. To do this we threshold the mask by setting all pixel values that are >= 0.5 to 1. All values < 0.5 we set to zero.


In [ ]:
y_pred=model1.predict(X_train)

In [ ]:
y_pred = (y_pred >= 0.5).astype(np.uint8)

We now plot the images to see the model results. 

In [ ]:
fig, (ax) = plt.subplots(figsize=(15,10), ncols=4, nrows=3)

for row_num in range(4):
    
    image_indx = np.random.randint(len(X_train))
    
    img = ax[0][row_num].imshow(X_train[image_indx][:,:,0],cmap='gray')
    ax[0][row_num].title.set_text('Needle_img')
    fig.colorbar(img, ax=ax[0][row_num])

    mask = ax[1][row_num].imshow(y_train[image_indx][:,:,0],cmap='gray')
    ax[1][row_num].title.set_text('Ground truth')
    fig.colorbar(mask, ax=ax[1][row_num])

    pred = ax[2][row_num].imshow(y_pred[image_indx][:,:,0],cmap='gray')
    ax[2][row_num].title.set_text('Needle_mask')
    fig.colorbar(pred, ax=ax[2][row_num])
    
plt.show()

The above images show the randomly picked needle-images, corresponding ground truth from triand data  and predicted needle-mask by the trained UNet model.

In [ ]:
y_pred_val=model1.predict(X_valid)

In [ ]:
y_pred_val = (y_pred_val >= 0.5).astype(np.uint8)

In [ ]:
fig, (ax) = plt.subplots(figsize=(15,10), ncols=4, nrows=3)

for row_num in range(4):
    
    image_indx = np.random.randint(len(X_valid))
    
    img = ax[0][row_num].imshow(X_valid[image_indx][:,:,0],cmap='gray')
    ax[0][row_num].title.set_text('Needle_img')
    fig.colorbar(img, ax=ax[0][row_num])

    mask = ax[1][row_num].imshow(y_valid[image_indx][:,:,0],cmap='gray')
    ax[1][row_num].title.set_text('Ground truth')
    fig.colorbar(mask, ax=ax[1][row_num])

    pred = ax[2][row_num].imshow(y_pred_val[image_indx][:,:,0],cmap='gray')
    ax[2][row_num].title.set_text('Needle_mask')
    fig.colorbar(pred, ax=ax[2][row_num])
    
plt.show()    

The above images show the randomly picked needle-images, corresponding ground truth from triand data  and predicted needle-mask by the trained UNet model.

# Prediction test data

In [ ]:
Y_pred_test =model1.predict(X_test)

In [ ]:
Y_pred_test = (Y_pred_test >= 0.5).astype(np.uint8)

In [ ]:
fig, (ax) = plt.subplots(figsize=(15,6), ncols=4, nrows=2)

for row_num in range(4):
    
    image_indx = np.random.randint(len(X_test))

    img = ax[0][row_num].imshow(X_test[image_indx][:,:,0],cmap='gray')
    ax[0][row_num].title.set_text('Needle_img')
    fig.colorbar(img, ax=ax[0][row_num])


    pred = ax[1][row_num].imshow(Y_pred_test[image_indx][:,:,0],cmap='gray')
    ax[1][row_num].title.set_text('Needle_mask')
    fig.colorbar(pred, ax=ax[1][row_num])

The above images show the randomly picked needle-images from test data and predicted needle-mask by the trained UNet model.

Leveraging the volumetric nature of the data through the inter-slice dependence of 2D slices is a key
factor in 3D biomedical image classification and segmentation problems.